In [1]:
import os
import sys

import numpy as np
import pandas as pd
import math

from trackml.dataset import load_event
from trackml.randomize import shuffle_hits
from trackml.score import score_event

sys.path.append("../..")
import merge as merge


In [2]:
TRAIN_PATH = '../../../input/train_1'
CSV_PATH = '../../good_csvs/'
event_id = 1003
CSV_FILENAME_PREFIX = 'event_' + str(event_id) + '_labels_train_'
CSV_HELIX1 = CSV_PATH + CSV_FILENAME_PREFIX + 'helix1.csv'
CSV_HELIX2 = CSV_PATH + CSV_FILENAME_PREFIX + 'helix2.csv'
CSV_HELIX3 = CSV_PATH + CSV_FILENAME_PREFIX + 'helix3.csv'
CSV_HELIX4 = CSV_PATH + CSV_FILENAME_PREFIX + 'helix4.csv'
CSV_HELIX5 = CSV_PATH + CSV_FILENAME_PREFIX + 'helix5.csv'
CSV_HELIX6 = CSV_PATH + CSV_FILENAME_PREFIX + 'helix6.csv'

In [3]:

event_prefix = 'event00000' + str(event_id)
hits, cells, particles, truth = load_event(os.path.join(TRAIN_PATH, event_prefix))

mem_bytes = (hits.memory_usage(index=True).sum() 
             + cells.memory_usage(index=True).sum() 
             + particles.memory_usage(index=True).sum() 
             + truth.memory_usage(index=True).sum())
print('{} memory usage {:.2f} MB'.format(event_prefix, mem_bytes / 2**20))

event000001003 memory usage 15.95 MB


In [4]:
labels_helix1 = pd.read_csv(CSV_HELIX1).label.values
labels_helix2 = pd.read_csv(CSV_HELIX2).label.values
labels_helix3 = pd.read_csv(CSV_HELIX3).label.values
#labels_helix4 = pd.read_csv(CSV_HELIX4).label.values
#labels_helix5 = pd.read_csv(CSV_HELIX5).label.values
#labels_helix6 = pd.read_csv(CSV_HELIX6).label.values

In [5]:
def create_one_event_submission(event_id, hits, labels):
    sub_data = np.column_stack(([event_id]*len(hits), hits.hit_id.values, labels))
    submission = pd.DataFrame(data=sub_data, columns=["event_id", "hit_id", "track_id"]).astype(int)
    return submission


In [6]:
labels_helix1_filter = merge.remove_outliers(labels_helix1, hits, print_counts=True)
labels_helix2_filter = merge.remove_outliers(labels_helix2, hits, print_counts=True)
labels_helix3_filter = merge.remove_outliers(labels_helix3, hits, print_counts=True)
#labels_helix4_filter = merge.remove_outliers(labels_helix4, hits, print_counts=True)
#labels_helix5_filter = merge.remove_outliers(labels_helix5, hits, print_counts=True)
#labels_helix6_filter = merge.remove_outliers(labels_helix6, hits, print_counts=True)

Total removed due to bad volumes: 0
Total removed due to bad dimensions: 0
Total removed due to duplicate zs: 268
Total removed due to bad slopes: 955
Total removed small tracks (<2) hits: 13698
Total removed due to bad volumes: 0
Total removed due to bad dimensions: 0
Total removed due to duplicate zs: 337
Total removed due to bad slopes: 710
Total removed small tracks (<2) hits: 14817
Total removed due to bad volumes: 0
Total removed due to bad dimensions: 0
Total removed due to duplicate zs: 503
Total removed due to bad slopes: 798
Total removed small tracks (<2) hits: 15156


In [7]:
labels_merged = merge.heuristic_merge_tracks(labels_helix1_filter, labels_helix2_filter)
labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix3_filter)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix4_filter)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix5_filter)
#labels_merged = merge.heuristic_merge_tracks(labels_merged, labels_helix6_filter)
one_submission = create_one_event_submission(event_id, hits, labels_merged)
score = score_event(truth, one_submission)
print("Merged score for event %d: %.8f" % (event_id, score))

Simple replacement of unclassified hits: 472
Similar tracks (no-op): 6048
Multiple non-trivial tracks: 4587
--> of which partial track ID 0 hits were updated: 2300
--> of which partial track ID non-0 hits were updated: 5913
Tracks to be lengthened: 2706
--> of which track ID 0 hits were updated: 3102
--> of which labels2 unique track lengths were: [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 20 23]
Simple replacement of unclassified hits: 216
Similar tracks (no-op): 5819
Multiple non-trivial tracks: 5430
--> of which partial track ID 0 hits were updated: 1866
--> of which partial track ID non-0 hits were updated: 4488
Tracks to be lengthened: 1934
--> of which track ID 0 hits were updated: 2037
--> of which labels2 unique track lengths were: [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Merged score for event 1003: 0.63221879
